## Introduction: 

This notebook will have all samples, including necrosis. We will use all of the samples with complete data, w/ an average of 76 samples to train on and 20 samples to test on. 

In [5]:
library(randomForestSRC)
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
library(randomForest)
#install.packages("Metrics")
library(Metrics)
source('AUC_workhorse.R')


 randomForestSRC 2.7.0 
 
 Type rfsrc.news() to see new features, changes, and bug fixes. 
 

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.


In [6]:
set.seed(101)
i = sample(1000, 1)

In [15]:
recgli = read.csv('../../../ParseData/7thParse_researchPath_withInVivo.csv')
recgli = recgli[which(recgli$include_all_advanced_mods==1), ]
recgli = recgli[which(!is.na(recgli$nt1v)),]



In [19]:
dim(recgli)
table(!duplicated(recgli$t_number))

[1] 104  75


FALSE  TRUE 
   43    61 

In [8]:
train_data = read.csv(train_path)
test_data = read.csv(test_path)

Warning message in file(file, "rt"):
“cannot open file '/home/sf673542/DataWrangling/AnalyzeData/AllBiopsies/Data/7thParse_TrainTestSplit/Training_withNec/train_data_train_data_373.csv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection


In [4]:
dim(train_data)
dim(test_data)

ERROR: Error in eval(expr, envir, enclos): object 'train_data' not found


In [6]:
data.frame(index = 1:length(colnames(train_data)), colnames = colnames(train_data))

index,colnames
1,b_number
2,t_number
3,sf_number
4,current_scan_date
5,current_surgery_date
6,current_hist_grade
7,current_hist_type
8,roi.label
9,tumor_cell_evaluation
10,necrosis


In [7]:
features_for_rf = colnames(train_data)[c(16:19, 21:22, 33, 34, 37:43, 46,66, 68,69)]
length(features_for_rf)
outcome = colnames(train_data)[9]

[1] 19

In [8]:
train_data = train_data[,c(outcome, features_for_rf)]
test_data = test_data[,c(outcome, features_for_rf)]

In [9]:
train_data = train_data[which(train_data$tumor_cell_evaluation != "Ind"),]
dim(train_data)
test_data = test_data[which(test_data$tumor_cell_evaluation != "Ind"),]
dim(test_data)

[1] 319  20

[1] 104  20

In [10]:
train_data = droplevels.data.frame(train_data)
test_data = droplevels.data.frame(test_data)

In [11]:
features.addsign <- paste(features_for_rf, collapse = "+")
rf.formula <- as.formula(paste(outcome,features.addsign, sep = "~"))

In [12]:
rf = rfsrc(formula = rf.formula,
     data = train_data, 
     ntree = 1000, 
     mtry = 5, 
     nodesize = 10)

In [13]:
print.rfsrc(rf)

                         Sample size: 76
           Frequency of class labels: NA, 14, 13, 20, 29
                     Number of trees: 1000
           Forest terminal node size: 10
       Average no. of terminal nodes: 7.094
No. of variables tried at each split: 5
              Total no. of variables: 19
       Resampling used to grow trees: swr
    Resample size used to grow trees: 76
                            Analysis: RF-C
                              Family: class
                      Splitting rule: gini *random*
       Number of random split points: 10
              Normalized brier score: 88.31 
                                 AUC: 56.81 
                          Error rate: 0.66, NA, 0.64, 1, 1, 0.28

Confusion matrix:

          predicted
  observed   0 1 2  3 class.error
           0 0 0 0  0         NaN
         0 0 5 0 5  4      0.6429
         1 0 2 0 2  9      1.0000
         2 0 3 1 0 16      1.0000
         3 0 3 0 5 21      0.2759

	Overall error rate: 65.79% 


In [25]:
preds = predict(rf, newdata=test_data)
preds

  Sample size of test (predict) data: 26
                Number of grow trees: 1000
  Average no. of grow terminal nodes: 6.664
         Total no. of grow variables: 19
       Resampling used to grow trees: swr
    Resample size used to grow trees: 72
                            Analysis: RF-C
                              Family: class
     Test set Normalized brier score: 103.11 
                        Test set AUC: 47.56 
                 Test set error rate: 0.77, 1, 1, 0.82, 0.5

Confusion matrix:

          predicted
  observed 0 1 2 3 class.error
         0 0 0 2 0      1.0000
         1 3 0 1 1      1.0000
         2 0 1 2 8      0.8182
         3 2 0 2 4      0.5000

	Overall error rate: 76.92% 

In [15]:
rf = rfsrc(formula = rf.formula,
     data = train_data, 
     ntree = 1000, 
     mtry = 5, 
     nodesize = 10)

[1] 0.5568783

In [16]:
auc_results = data.frame('train_test_number'=1:1000, 'AUC'=NA)

In [22]:
for (i in c(1:11, 13:19, 21:1000)){
    print(i)
    ## define train and test paths
    train_path = paste('../../AllBiopsies/Training_withNec/train_data_',as.character(i),'.csv', sep = "")
    test_path = paste('../../AllBiopsies/Testing_withNec/test_data_',as.character(i),'.csv', sep = "")

    ## read in data
    train_data = read.csv(train_path)
    test_data = read.csv(test_path)

    ## remove "ind" pathology and empty string
    train_data = train_data[which(train_data$tumor_cell_evaluation != "Ind"),]
    train_data = train_data[which(train_data$tumor_cell_evaluation != ""),]
    test_data = test_data[which(test_data$tumor_cell_evaluation != "Ind"),]
    test_data = test_data[which(test_data$tumor_cell_evaluation != ""),]

    ## droplevels: 
    train_data$tumor_cell_evaluation = droplevels(as.factor(train_data$tumor_cell_evaluation))
    test_data$tumor_cell_evaluation = droplevels(as.factor(test_data$tumor_cell_evaluation))
    
    ## define features: 
    features_for_rf = colnames(train_data)[c(16:19, 21:22, 33, 34, 37:43, 46,66, 68,69)]
    outcome = colnames(train_data)[9]
    
    ## create formula 
    features.addsign <- paste(features_for_rf, collapse = "+")
    rf.formula <- as.formula(paste(outcome,features.addsign, sep = "~"))
    
    ## run random forest experiment
    rf = rfsrc(formula = rf.formula,
     data = train_data, 
     ntree = 1000, 
     mtry = 5, 
     nodesize = 10)
    
    ## predict on the test set 
    preds = predict(rf, newdata=test_data)
    
    ## find the AUC of the classification: 
    x = preds
    aucS <- auc(x$yvar, if(!is.null(x$predicted.oob) && !all(is.na(x$predicted.oob))) x$predicted.oob else x$predicted)
    
    ## add the results to a data_frame 
    auc_results[i, 2]=aucS
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 139
[1] 140
[1

In [24]:
summary(auc_results[,2])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 0.2708  0.5267  0.5850  0.5831  0.6391  0.8347       2 

## Experiment 2: 

In [26]:
for (i in c(1:11, 13:19, 21:1000)){
    print(i)
    ## define train and test paths
    train_path = paste('../../AllBiopsies/Training_withNec/train_data_',as.character(i),'.csv', sep = "")
    test_path = paste('../../AllBiopsies/Testing_withNec/test_data_',as.character(i),'.csv', sep = "")

    ## read in data
    train_data = read.csv(train_path)
    test_data = read.csv(test_path)

    ## remove "ind" pathology and empty string
    train_data = train_data[which(train_data$tumor_cell_evaluation != "Ind"),]
    train_data = train_data[which(train_data$tumor_cell_evaluation != ""),]
    test_data = test_data[which(test_data$tumor_cell_evaluation != "Ind"),]
    test_data = test_data[which(test_data$tumor_cell_evaluation != ""),]

    ## droplevels: 
    train_data$tumor_cell_evaluation = droplevels(as.factor(train_data$tumor_cell_evaluation))
    test_data$tumor_cell_evaluation = droplevels(as.factor(test_data$tumor_cell_evaluation))
    
    ## define features: 
    features_for_rf = colnames(train_data)[c(16:19, 21:22, 33, 34, 37:43, 46,66, 68,69)]
    outcome = colnames(train_data)[9]
    
    ## create formula 
    features.addsign <- paste(features_for_rf, collapse = "+")
    rf.formula <- as.formula(paste(outcome,features.addsign, sep = "~"))
    
    ## run random forest experiment
    rf = rfsrc(formula = rf.formula,
     data = train_data, 
     ntree = 1000, 
     mtry = 5, 
     nodesize = 10)
    
    ## predict on the test set 
    preds = predict(rf, newdata=test_data)
    
    ## find the AUC of the classification: 
    x = preds
    aucS <- auc(x$yvar, if(!is.null(x$predicted.oob) && !all(is.na(x$predicted.oob))) x$predicted.oob else x$predicted)
    
    ## add the results to a data_frame 
    auc_results[i, 2]=aucS
}

[1] "nfse"      "nfl"       "nt1c"      "nt1v"      "nadc.1"    "nfa.1"    
 [7] "cbvn_nlin" "phn_nlin"  "cni"       "ccri"      "crni"      "ncho"     
[13] "ncre"      "nnaa"      "laclip"    "olddata"   "in_CEL"    "in_NEL"   
[19] "in_NEC"